In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import numpy as np # linear algebra
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image, ImageFilter, ImageEnhance # dealing with images 
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import os
        
directory = '/kaggle/input/early-detection-of-3d-printing-issues'
df_train = pd.read_csv(directory + '/train.csv')
df_test = pd.read_csv(directory + '/test.csv')

df_train_0 = df_train[df_train['has_under_extrusion'] == 0].sample(3672) 
df_train_1 = df_train[df_train['has_under_extrusion'] == 1].sample(5029) 

img_path = '/kaggle/working/training_images/0'
os.makedirs(img_path)

for index in range(df_train_0['img_path'].count()):
    image = Image.open(directory + '/images/' + df_train_0['img_path'].iloc[index])
    string_img_name = df_train_0['img_path'].iloc[index]
    string_img_name = string_img_name.replace("/", "-")
    image = image.save(img_path + "/" + string_img_name + '.jpg')
    
img_path = '/kaggle/working/training_images/1'
os.makedirs(img_path)

for index in range(df_train_1['img_path'].count()):
    image = Image.open(directory + '/images/' + df_train_1['img_path'].iloc[index])
    string_img_name = df_train_1['img_path'].iloc[index]
    string_img_name = string_img_name.replace("/", "-")
    image = image.save(img_path + "/" + str(index + 1) + '.jpg')

train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [2]:
img_path = '/kaggle/working/validation_images/'
labels = []
os.makedirs(img_path + "1/")
os.makedirs(img_path + "0/")

count = 0 
while count < 2176:
    sample = df_train.sample(1)
    label = sample['has_under_extrusion'].iloc[0]
    name = sample['img_path'].iloc[0]
    dashed_name = name.replace("/","-")
    try: 
        image = Image.open(img_path + str(label) + "/" + dashed_name)
    except FileNotFoundError:
        try:
            image = Image.open('/kaggle/working/training_images/' + str(label) + "/" + dashed_name)
        except FileNotFoundError:
            image = Image.open(directory + '/images/' + name)
            image = image.save(img_path + str(label) + "/" + dashed_name)
            labels.append(label)
            count += 1
    

In [3]:
training_data = train.flow_from_directory("/kaggle/working/training_images",
                                         target_size = (255,255),
                                         batch_size = 16, 
                                         class_mode = 'binary')

Found 8701 images belonging to 2 classes.


In [4]:
validation_data = validation.flow_from_directory("/kaggle/working/validation_images",
                                                target_size = (255,255),
                                                batch_size = 16,
                                                class_mode = 'binary')

Found 2176 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (5,5), activation = "relu", input_shape = (255,255,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [6]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam', 
    metrics = ['accuracy'], 
    
)

In [7]:
model_fit = model.fit(
    training_data,
    steps_per_epoch = 3, 
    epochs = 20, 
    validation_data = validation_data
)

Epoch 1/20
3/3 [==============================] - 45s 21s/step - loss: 1.3438 - accuracy: 0.5625 - val_loss: 0.9376 - val_accuracy: 0.4366
Epoch 2/20
3/3 [==============================] - 42s 20s/step - loss: 0.8010 - accuracy: 0.4792 - val_loss: 0.6389 - val_accuracy: 0.5653
Epoch 3/20
3/3 [==============================] - 43s 21s/step - loss: 0.6472 - accuracy: 0.5625 - val_loss: 0.6178 - val_accuracy: 0.5970
Epoch 4/20
3/3 [==============================] - 43s 21s/step - loss: 0.6098 - accuracy: 0.6250 - val_loss: 0.6029 - val_accuracy: 0.6434
Epoch 5/20
3/3 [==============================] - 42s 21s/step - loss: 0.6074 - accuracy: 0.6222 - val_loss: 0.5749 - val_accuracy: 0.6985
Epoch 6/20
3/3 [==============================] - 43s 21s/step - loss: 0.5348 - accuracy: 0.7708 - val_loss: 0.5483 - val_accuracy: 0.6834
Epoch 7/20
3/3 [==============================] - 42s 21s/step - loss: 0.5513 - accuracy: 0.6667 - val_loss: 0.6042 - val_accuracy: 0.6278
Epoch 8/20
3/3 [===========

In [ ]:
hue_list = []

for index in range(df_test['img_path'].count()):
    
    image = Image.open(directory + '/images/' + df_test['img_path'].iloc[index])
    image = image.resize((255,255))
    img_arr = np.asarray(image)
    img_arr = np.expand_dims(img_arr, axis = 0)
    value = model.predict(img_arr)
    hue_list.append(value[0][0])
    
hue_list1 = [round(x) for x in hue_list]
df_test['has_under_extrusion'] = hue_list1
df_final = df_test[['img_path', 'has_under_extrusion']]
df_final.to_csv('/kaggle/working/validation_test1.csv', index = False)    

1/1 [==============================] - 0s 41ms/step
